# Feature Extraction

In [4]:
import os
import sys
import json
import numpy as np
import pandas as pd

In [5]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

In [6]:
columns = read_json('../orderdListGenres.json')
print(len(columns), columns)

20 ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']


In [7]:
SAVE_NAME = 'test'

In [ ]:
from keras.models import Model

from keras.preprocessing.text import Tokenizer


## Reload model

In [2]:
WEIGHTS_LOAD = './weight/merge_stride4.h5'

from keras.models import load_model

model = load_model(WEIGHTS_LOAD)

C:\Users\TONYLAB\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\TONYLAB\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\TONYLAB\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Us

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 330)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 330, 300)     5414100     input_4[0][0]                    
__________________________________________________________________________________________________
reshape_7 (Reshape)             (None, 330, 300, 1)  0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 328, 1, 100)  90100       reshape_7[0][0]                  
__________________________________________________________________________________________________
conv2d_11 

In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 330)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 330, 300)     5414100     input_4[0][0]                    
__________________________________________________________________________________________________
reshape_7 (Reshape)             (None, 330, 300, 1)  0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 328, 1, 100)  90100       reshape_7[0][0]                  
__________________________________________________________________________________________________
conv2d_11 

## Testing

In [8]:
sequences_test=tokenizer.texts_to_sequences(test_data.convert_text)
X_test = pad_sequences(sequences_test,maxlen=X_train.shape[1])
pred = model.predict(X_test)
pred.shape

NameError: name 'tokenizer' is not defined

In [ ]:
predictions = pred

results=pd.DataFrame(predictions, columns=columns)
results["username"] = test_data.username
ordered_cols = ["username"] + columns
results = results[ordered_cols] #To get the same column order
results.to_csv("./output/"+ SAVE_NAME + "_result.csv",index=False)

In [ ]:
labels = read_json('../genresDic.json')

In [ ]:
def countAcc(predList, trueList):
    fraction = 0
    denominator = len(trueList)
    for g in predList:
        if g in trueList:
            fraction += 1 
    return fraction / denominator

## Label top n dynamic

In [ ]:
true_df = pd.read_csv('./input/true_df.csv')
true_df

In [ ]:
results = pd.read_csv("./output/"+ SAVE_NAME + "_result.csv")
results

In [ ]:
li = []
for i, row in true_df.iterrows():
    username = row['username']
    if username == results.loc[i,:]['username']: # username equals
        print(i, username)
        truth = labels[username]
        numTrue = sum(row[1:])
        print('count:', numTrue)
        
        s = results.loc[i,:][1:]
        top_n = s.sort_values(ascending=False)[:numTrue]
        inList = list(top_n.index)
        
        onehot = []
        for g in columns:
            onehot.append(int(g in inList))
            
        acc = countAcc(inList, truth)
        print(acc, inList, truth)
        
        appList = [username, acc, truth, inList] + onehot
        li.append(appList)

In [ ]:
acc_df = pd.DataFrame(li, columns = ['username', 'acc', 'truth', 'top_n'] + columns)
acc_df

In [ ]:
acc_df.to_csv("./output/"+ SAVE_NAME + "_bi.csv", index = 0)

In [ ]:
acc_df['acc'].mean()

## Feature Extraction

In [ ]:
extract_layer = Model(inputs=model.input,
                      outputs=model.get_layer('flatten_1').output)
#以这个model的预测值作为输出
extract_output = extract_layer.predict(X_train)
 
print(extract_output.shape)
# print(extract_output[0])